<a href="https://colab.research.google.com/github/travisngt/ML_Breast_Cancer_Final_Project/blob/main/DL_(Yolo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the annotation file
annotations_path = '/content/drive/My Drive/breast_cancer_annotations/final_breast_cancer_annotations14.csv'
annotations = pd.read_csv(annotations_path)

# Define the image dimensions (assuming all images are of the same size)
image_width = 1024
image_height = 1024

# Create directory to save YOLO labels in Google Drive
yolo_labels_dir = '/content/drive/My Drive/yolo_labels1'
os.makedirs(yolo_labels_dir, exist_ok=True)

# Process annotations to create YOLO label files
for index, row in annotations.iterrows():
    # Skip rows without abnormality information
    if pd.isna(row['X_Center']) or pd.isna(row['Y_Center']) or pd.isna(row['Radius']):
        continue

    # Extract class and normalize coordinates
    class_id = row['Label']  # Assume Label: 0 = Normal, 1 = Benign, 2 = Malignant
    x_center = row['X_Center'] / image_width
    y_center = row['Y_Center'] / image_height
    width = (2 * row['Radius']) / image_width
    height = (2 * row['Radius']) / image_height

    # Ensure coordinates are within the bounds [0, 1]
    x_center = min(max(x_center, 0), 1)
    y_center = min(max(y_center, 0), 1)
    width = min(max(width, 0), 1)
    height = min(max(height, 0), 1)

    # Create YOLO label file
    image_id = row['Image_ID']
    label_file_path = os.path.join(yolo_labels_dir, f"{image_id}.txt")
    with open(label_file_path, 'w') as f:
        f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    print(f"Created label file: {label_file_path}")  # Debugging message

print(f"YOLO labels have been saved to: {yolo_labels_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created label file: /content/drive/My Drive/yolo_labels1/mdb001.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb002.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb003.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb004.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb005.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb005.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb006.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb007.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb008.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb009.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb010.txt
Created label file: /content/drive/My Drive/yolo_labels1/mdb011.txt
Created label file: /content/drive/My Drive/yolo_labels

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Đường dẫn gốc ảnh và nhãn
images_dir = '/content/drive/My Drive/images_processed/'  # Thư mục chứa các thư mục con
yolo_labels_dir = '/content/drive/My Drive/yolo_labels1/'  # Thư mục nhãn YOLO

# Thư mục lưu trữ train/val/test
output_dir = '/content/drive/My Drive/yolo_dataset1'
os.makedirs(output_dir, exist_ok=True)
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)

# Duyệt qua các thư mục con để tìm ảnh
image_paths = {}
for class_dir in os.listdir(images_dir):
    full_class_dir = os.path.join(images_dir, class_dir)
    if os.path.isdir(full_class_dir):  # Chỉ xử lý thư mục
        for img_file in os.listdir(full_class_dir):
            if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Hỗ trợ nhiều định dạng ảnh
                image_id = os.path.splitext(img_file)[0]  # Tên file không bao gồm đuôi
                image_paths[image_id] = os.path.join(full_class_dir, img_file)

# Lấy danh sách nhãn và khớp với ảnh
label_files = os.listdir(yolo_labels_dir)
matched_images = []
matched_labels = []

for label_file in label_files:
    image_id = os.path.splitext(label_file)[0]
    label_path = os.path.join(yolo_labels_dir, label_file)
    if image_id in image_paths:  # Nếu tìm thấy ảnh tương ứng
        matched_images.append(image_paths[image_id])
        matched_labels.append(label_path)
    else:
        print(f"Không tìm thấy ảnh cho nhãn: {label_file}")

# Phân chia train/val/test
train_images, val_test_images, train_labels, val_test_labels = train_test_split(
    matched_images, matched_labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(
    val_test_images, val_test_labels, test_size=0.5, random_state=42)

# Hàm sao chép ảnh và nhãn
def copy_files(images, labels, split):
    for img_path, lbl_path in zip(images, labels):
        dst_img_path = os.path.join(output_dir, 'images', split, os.path.basename(img_path))
        dst_lbl_path = os.path.join(output_dir, 'labels', split, os.path.basename(lbl_path))

        # Kiểm tra tồn tại file ảnh và nhãn trước khi sao chép
        if not os.path.exists(img_path):
            print(f"Ảnh không tồn tại: {img_path}. Bỏ qua.")
            continue
        if not os.path.exists(lbl_path):
            print(f"Nhãn không tồn tại: {lbl_path}. Bỏ qua.")
            continue

        shutil.copy(img_path, dst_img_path)
        shutil.copy(lbl_path, dst_lbl_path)

# Sao chép file
copy_files(train_images, train_labels, 'train')
copy_files(val_images, val_labels, 'val')
copy_files(test_images, test_labels, 'test')

# Tổng hợp thông tin
print(f"Tổng số ảnh và nhãn khớp: {len(matched_images)}")
print(f"Tập train: {len(train_images)} ảnh, {len(train_labels)} nhãn")
print(f"Tập val: {len(val_images)} ảnh, {len(val_labels)} nhãn")
print(f"Tập test: {len(test_images)} ảnh, {len(test_labels)} nhãn")
print(f"Dataset đã được lưu tại: {output_dir}")


Không tìm thấy ảnh cho nhãn: mdb059.txt
Không tìm thấy ảnh cho nhãn: aug_1_mdb023.png.txt
Không tìm thấy ảnh cho nhãn: aug_2_mdb028.png.txt
Không tìm thấy ảnh cho nhãn: aug_3_mdb058.png.txt
Không tìm thấy ảnh cho nhãn: aug_4_mdb072.png.txt
Không tìm thấy ảnh cho nhãn: aug_5_mdb075.png.txt
Không tìm thấy ảnh cho nhãn: aug_6_mdb090.png.txt
Không tìm thấy ảnh cho nhãn: aug_7_mdb092.png.txt
Không tìm thấy ảnh cho nhãn: aug_8_mdb095.png.txt
Không tìm thấy ảnh cho nhãn: aug_9_mdb102.png.txt
Không tìm thấy ảnh cho nhãn: aug_10_mdb105.png.txt
Không tìm thấy ảnh cho nhãn: aug_11_mdb110.png.txt
Không tìm thấy ảnh cho nhãn: aug_12_mdb111.png.txt
Không tìm thấy ảnh cho nhãn: aug_13_mdb115.png.txt
Không tìm thấy ảnh cho nhãn: aug_14_mdb117.png.txt
Không tìm thấy ảnh cho nhãn: aug_15_mdb120.png.txt
Không tìm thấy ảnh cho nhãn: aug_16_mdb124.png.txt
Không tìm thấy ảnh cho nhãn: aug_17_mdb125.png.txt
Không tìm thấy ảnh cho nhãn: aug_18_mdb130.png.txt
Không tìm thấy ảnh cho nhãn: aug_19_mdb134.png.txt


In [ ]:
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(output_dir, 'images', split)
    lbl_dir = os.path.join(output_dir, 'labels', split)
    print(f"{split.capitalize()} - Images: {len(os.listdir(img_dir))}, Labels: {len(os.listdir(lbl_dir))}")


Train - Images: 224, Labels: 224
Val - Images: 48, Labels: 48
Test - Images: 49, Labels: 49


In [ ]:
import yaml

# Nội dung file data.yaml
data = {
    'train': '/content/drive/My Drive/yolo_dataset1/images/train',
    'val': '/content/drive/My Drive/yolo_dataset1/images/val',
    'nc': 3,
    'names': ['Normal', 'Benign', 'Malignant']
}

# Đường dẫn lưu file data.yaml
data_yaml_path = '/content/drive/My Drive/yolo_dataset1/data.yaml'

# Ghi nội dung vào file data.yaml
with open(data_yaml_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)

print(f"File data.yaml has been created at: {data_yaml_path}")


File data.yaml has been created at: /content/drive/My Drive/yolo_dataset1/data.yaml


In [ ]:
!pip install ultralytics
from ultralytics import YOLO



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')
# Load YOLOv8 model và huấn luyện
model = YOLO('yolov8s.pt')  # Tải mô hình YOLOv8 nhỏ
model.train(data='/content/drive/My Drive/yolo_dataset1/data.yaml', epochs=100, imgsz=1024)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/My Drive/yolo_dataset1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, reti

100%|██████████| 755k/755k [00:00<00:00, 20.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 91.1MB/s]


AMP: checks passed ✅


...

                   all         48         48    0.00277      0.322     0.0119    0.00529



100 epochs completed in 0.435 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 22.6MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.06it/s]


                   all         48         48    0.00471      0.458     0.0222    0.00487
                Normal         30         30     0.0119        0.9     0.0637     0.0142
                Benign         10         10    0.00141        0.1    0.00198   0.000198
             Malignant          8          8   0.000834      0.375    0.00077   0.000227
Speed: 0.5ms preprocess, 11.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f212446fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04